In [3]:
import keras.utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


2023-10-11 13:14:31.513080: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 13:14:32.084765: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 13:14:32.087735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 13:14:34.007364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
label_encoder = LabelEncoder()
data = pd.read_csv('combined_data.csv')

# Split the data into features (X) and labels (y)
X = data['Nhận xét đánh giá'].values
y = data['Cảm xúc'].values

# Encode the labels
label_encoder = LabelEncoder()
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Mã hóa nhãn thành dạng số
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [8]:

max_words = 5000  # Số từ tối đa được xem xét
max_len = 1000  # Độ dài tối đa của một chuỗi

# Tokenize
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_train_seq = keras.utils.pad_sequences(X_train_seq, maxlen=max_len)
X_val_seq = keras.utils.pad_sequences(X_val_seq, maxlen=max_len)
embedding_dim = 120  # Kích thước của embedding cho từ
num_filters = 128  # Số lượng bộ lọc trong lớp tích chập
kernel_size = 5
num_classes = len(label_encoder.classes_)  # Số lượng lớp đầu ra
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Conv1D(num_filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:

batch_size = 64
epochs = 10

model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_seq, y_val))

2023-10-11 13:20:08.308804: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 61152000 exceeds 10% of free system memory.


Epoch 1/10


2023-10-11 13:20:09.918409: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2023-10-11 13:20:09.929448: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32636928 exceeds 10% of free system memory.
2023-10-11 13:20:10.114589: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32636928 exceeds 10% of free system memory.
2023-10-11 13:20:10.178039: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23904000 exceeds 10% of free system memory.


101/239 [===========>..................] - ETA: 59s - loss: 1.4256 - accuracy: 0.4553

KeyboardInterrupt: 

In [7]:

loss, accuracy = model.evaluate(X_val_seq, y_val)
print(f"Mất mát trên tập xác thực: {loss}")
print(f"Độ chính xác trên tập xác thực: {accuracy}")


120/120 [==============================] - 6s 47ms/step - loss: 0.9682 - accuracy: 0.6864
Mất mát trên tập xác thực: 0.9681808948516846
Độ chính xác trên tập xác thực: 0.6863719820976257


In [8]:
y_pred = model.predict(X_val_seq)
# Convert numerical labels to original labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_labels)

# Convert true labels to original labels
y_val_labels = label_encoder.inverse_transform(y_val)

# Generate classification report
report = classification_report(y_val_labels, y_pred_labels)
print(report)

120/120 [==============================] - 7s 57ms/step
                 precision    recall  f1-score   support

    bình thường       0.64      0.59      0.61       974
không liên quan       0.96      0.95      0.96       218
         rất tệ       0.77      0.73      0.75       263
        rất tệ        0.39      0.22      0.29        49
        rất tốt       0.67      0.58      0.62       672
             tệ       0.88      0.82      0.85       769
            tốt       0.55      0.71      0.62       878

       accuracy                           0.69      3823
      macro avg       0.69      0.66      0.67      3823
   weighted avg       0.70      0.69      0.69      3823



In [11]:
custom_text = "rất tuyệt vời"

# Tokenize and pad the custom text
custom_text_seq = tokenizer.texts_to_sequences([custom_text])
custom_text_seq = keras.utils.pad_sequences(custom_text_seq, maxlen=max_len)

# Make predictions on the custom text
custom_text_pred = model.predict(custom_text_seq)

# Convert numerical labels to original labels
custom_text_pred_label = np.argmax(custom_text_pred, axis=1)
custom_text_pred_label = label_encoder.inverse_transform(custom_text_pred_label)

# Print the predicted label
print(f"Predicted Sentiment: {custom_text_pred_label[0]}")

1/1 [==============================] - 0s 25ms/step
Predicted Sentiment: tốt


In [13]:
model.save('CNN_UIT.h5')